In [1]:
#!/usr/bin/env python
# coding: utf-8

import rosbag, os, matplotlib, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from qsrlib.qsrlib import QSRlib, QSRlib_Request_Message
from qsrlib_io.world_trace import Object_State, World_Trace
from qsrlib.qsrlib import QSR_QTC_BC_Simplified
import numpy as np
import pandas as pd
import datetime as dt

os.chdir("/home/loz/QTC_Trajectory_HMMs/from_bags/")

In [2]:
# In[2]:

lab_bags = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("~/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags")) for f in fn]
lab_bags

['/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/7/pp7.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/3/pp3.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/10/pp10.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/12/pp12.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/8/pp8.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/11/pp11.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/4/pp4.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/6/pp6.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/2/pp2.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/1/pp1.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/pbl/5/pp5.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/rotr/7/pp7.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/study_HRSI_rosbags/rotr/3/p

In [3]:
# In[60]:


r_positions = []
h_positions = []

r_state_seqs = []
h_state_seqs = []

qsrlib = QSRlib()

for bag_path in lab_bags:
    bag = rosbag.Bag(bag_path)
    
    r_xs = []
    r_ys = []
    
    r_state_seq = []
    for topic, msg, t in bag.read_messages(topics=['/robot5/control/odom']):
        t = t.to_sec()
        x = msg.pose.pose.position.x
        y = msg.pose.pose.position.y
        
        r_xs.append(x)
        r_ys.append(y)
        
        r_state_seq.append(Object_State(name="robot", timestamp=t, x=x, y=y))
    r_state_seqs.append(r_state_seq)
    
    h_xs = []
    h_ys = []
    
    h_state_seq = []
    for topic, msg, t in bag.read_messages(topics=['/robot5/people_tracker_filtered/positions']):
        t = t.to_sec()
        try:
            x = msg.poses[0].position.x
            y = msg.poses[0].position.y
            
            h_xs.append(x)
            h_ys.append(y)
            
            h_state_seq.append(Object_State(name="human", timestamp=t, x=x, y=y))
    
        except:
            pass
    h_state_seqs.append(h_state_seq)
    
    bag.close()
    r_positions.append([r_xs, r_ys])
    h_positions.append([h_xs, h_ys])


# In[61]:

In [4]:
# Test getting QTC_C sequence
bag_no = 0
quantisation_factor = 0.01

world = World_Trace()

h_x = [h_state_seqs[bag_no][i].x for i in range(len(h_state_seqs[bag_no]))]
h_y = [h_state_seqs[bag_no][i].y for i in range(len(h_state_seqs[bag_no]))]


r_x = [r_state_seqs[bag_no][i].x for i in range(len(r_state_seqs[bag_no]))]
r_y = [r_state_seqs[bag_no][i].y for i in range(len(r_state_seqs[bag_no]))]

In [5]:
# In[62]:


# Downsample state series' to 200kHz frequency
h_state_series = pd.DataFrame({"x": h_x, "y": h_y},
    index=[pd.to_datetime(h_state_seqs[bag_no][i].timestamp, unit="s") for i in range(len(h_state_seqs[bag_no]))])

h_state_series = h_state_series.resample("200ms").mean()

In [6]:
# In[63]:


r_state_series = pd.DataFrame({"x": r_x, "y": r_y},
    index=[pd.to_datetime(r_state_seqs[bag_no][i].timestamp, unit="s") for i in range(len(r_state_seqs[bag_no]))])

r_state_series = r_state_series.resample("200ms").mean()

In [7]:
# In[64]:


# Create world_trace state series from downsampled human position data
h_state_seq = []
for index, row in h_state_series.iterrows():
    x = row['x']
    y = row['y']
    t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()
    
    h_state_seq.append(Object_State(name="human", timestamp=t, x=x, y=y))

In [8]:
# In[65]:


# Create world_trace state series from downsampled robot position data
r_state_seq = []
for index, row in r_state_series.iterrows():
    x = row['x']
    y = row['y']
    t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()
    
    r_state_seq.append(Object_State(name="robot", timestamp=t, x=x, y=y))

In [9]:
# In[83]:


world.add_object_state_series(h_state_seq)
world.add_object_state_series(r_state_seq)

# make a QSRlib request message
dynamic_args = {"qtccs": {"no_collapse": True, "quantisation_factor": quantisation_factor,
                                        "validate": False, "qsrs_for": [("human", "robot")]}}

qsrlib_request_message = QSRlib_Request_Message(
    'qtccs', world, dynamic_args)

# request your QSRs
qsrlib_response_message = qsrlib.request_qsrs(req_msg=qsrlib_request_message)
qsrlib_response_message

/home/loz/QTC_Trajectory_HMMs/from_bags/qtcs_from_bags_py2/venv/local/lib/python2.7/site-packages/numpy/linalg/linalg.py:2093: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [10]:
# Get QSR at each timestamp
timestamps = qsrlib_response_message.qsrs.get_sorted_timestamps()
for t in timestamps:
    for val in qsrlib_response_message.qsrs.trace[t].qsrs.values():
        print(val.qsr['qtccs'].replace(",",""))

----
----
----
-0--
+0--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
++--
0+0-
0000
0000
0000
0000
0000
0000
-+-0
-+-0


In [11]:
# In[9]:
bag_path = lab_bags[bag_no]
# bag_path[67:].replace("/", "_")[:-4]
"_".join(bag_path[59:].replace("/", "_")[:-4].split("_")[:2])

'pbl_7'

# Build dict of bags and their QTC_C sequences

In [12]:
quantisation_factor = 0.01
qtc_seqs = {}

for bag_no in range(len(lab_bags)):
    qtc_seq = []
    bag_path = lab_bags[bag_no]
    sit_code = "_".join(bag_path[59:].replace("/", "_")[:-4].split("_")[:2])
    initial_bag_nos = range(1,6)
    
    print(sit_code)
    
    world = World_Trace()

    h_x = [h_state_seqs[bag_no][i].x for i in range(len(h_state_seqs[bag_no]))]
    h_y = [h_state_seqs[bag_no][i].y for i in range(len(h_state_seqs[bag_no]))]


    r_x = [r_state_seqs[bag_no][i].x for i in range(len(r_state_seqs[bag_no]))]
    r_y = [r_state_seqs[bag_no][i].y for i in range(len(r_state_seqs[bag_no]))]
    


    # Downsample state series' to 200kHz frequency
    h_state_series = pd.DataFrame({"x": h_x, "y": h_y},
        index=[pd.to_datetime(h_state_seqs[bag_no][i].timestamp, unit="s") for i in range(len(h_state_seqs[bag_no]))])

    h_state_series = h_state_series.resample("200ms").mean()

    
    r_state_series = pd.DataFrame({"x": r_x, "y": r_y},
        index=[pd.to_datetime(r_state_seqs[bag_no][i].timestamp, unit="s") for i in range(len(r_state_seqs[bag_no]))])

    r_state_series = r_state_series.resample("200ms").mean()
    
    # Correct for erroneous odometry tracking of robot position
    r_x_start = r_state_series.x.values[0]
    r_x_end = r_state_series.x.values[-1]
    r_y_start = r_state_series.y.values[0]
    r_y_end = r_state_series.y.values[-1]

    r_start_end_dist = np.sqrt(((r_x_end - r_x_start)**2) + ((r_y_end - r_y_start)**2))

    if r_start_end_dist > 1:
        r_state_series.y = np.linspace(2, 3.5, len(r_state_series.x.values))
        r_state_series.x = np.linspace(1, 4.5, len(r_state_series.y.values))
    else:
        r_state_series.y = np.linspace(2, 2, len(r_state_series.x.values))
        r_state_series.x = np.linspace(1, 1, len(r_state_series.y.values))
    
    with open("lab_sit_starts_ends.pickle", "r") as f:
        starts_ends_ts = pickle.load(f)
    
#     if int(sit_code.split("_")[-1]) in initial_bag_nos:
#         h_state_series = h_state_series.loc[starts_ends_ts[sit_code][0]:starts_ends_ts[sit_code][1]]
#         r_state_series = r_state_series.loc[starts_ends_ts[sit_code][0]:starts_ends_ts[sit_code][1]]
    
    start = max(r_state_series.index.min(), h_state_series.index.min())
    end = min(r_state_series.index.max(), h_state_series.index.max())

    h_state_series = h_state_series.resample("200ms").interpolate()
    r_state_series = r_state_series.resample("200ms").interpolate()
    
    r_state_series = r_state_series.loc[start:end]
    h_state_series = h_state_series.loc[start:end]

    # Correct for frame for people positions being robot position
    h_state_series.x = [h_state_series.x.values[i] + r_state_series.x.values[i] for i in range(len(h_state_series.x.values))]
    h_state_series.y = [h_state_series.y.values[i] + r_state_series.y.values[i] for i in range(len(h_state_series.y.values))]
    

    # Create world_trace state series from downsampled human position data
    h_state_seq = []
    for index, row in h_state_series.iterrows():
        x = row['x']
        y = row['y']
        t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()

        h_state_seq.append(Object_State(name="human", timestamp=t, x=x, y=y))
        
        
    # Create world_trace state series from downsampled robot position data
    r_state_seq = []
    for index, row in r_state_series.iterrows():
        x = row['x']
        y = row['y']
        t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()

        r_state_seq.append(Object_State(name="robot", timestamp=t, x=x, y=y))
        

    # Add human and robot trajectories to world
    world.add_object_state_series(h_state_seq)
    world.add_object_state_series(r_state_seq)

    # make a QSRlib request message
    dynamic_args = {"qtccs": {"no_collapse": False, "quantisation_factor": quantisation_factor,
                                            "validate": False, "qsrs_for": [("robot", "human")]}}

    qsrlib_request_message = QSRlib_Request_Message(
        'qtccs', world, dynamic_args)

    # request your QSRs
    qsrlib_response_message = qsrlib.request_qsrs(req_msg=qsrlib_request_message)
    qsrlib_response_message
    
    
    # Get QSR at each timestamp
    timestamps = qsrlib_response_message.qsrs.get_sorted_timestamps()
    for t in timestamps:
        for val in qsrlib_response_message.qsrs.trace[t].qsrs.values():
            qtc_seq.append(val.qsr['qtccs'].replace(",",""))
    qtc_seqs[sit_code] = qtc_seq

pbl_7
pbl_3
pbl_10
pbl_12
pbl_8
pbl_11
pbl_4
pbl_6
pbl_2
pbl_1
pbl_5
rotr_7
rotr_3
rotr_10
rotr_8
rotr_11
rotr_4
rotr_6
rotr_2
rotr_1
rotr_5
pcl_7
pcl_3
pcl_10
pcl_12
pcl_8
pcl_11
pcl_4
pcl_6
pcl_2
pcl_1
pcl_5
pbr_7
pbr_3
pbr_10
pbr_12
pbr_8
pbr_11
pbr_4
pbr_6
pbr_2
pbr_1
pbr_5
pcr_7
pcr_3
pcr_10
pcr_8
pcr_11
pcr_4
pcr_6
pcr_2
pcr_1
pcr_5
rotl_7
rotl_3
rotl_10
rotl_12
rotl_8
rotl_11
rotl_4
rotl_6
rotl_2
rotl_1
rotl_5
u_7
u_3
u_10
u_12
u_8
u_11
u_4
u_6
u_2
u_1
u_5


In [13]:
qtc_seqs

{'pbl_1': ['----',
  '0---',
  '+---',
  '+0--',
  '++--',
  '++0-',
  '+++-',
  '+-+-',
  '+-+0',
  '+-++',
  '+-+0'],
 'pbl_10': ['----', '0---', '+---', '+0--', '++--', '++0-', '+++-', '+-+-'],
 'pbl_11': ['----',
  '0---',
  '+0--',
  '++--',
  '++0-',
  '+-0-',
  '+-+-',
  '+-++',
  '+-+0',
  '+-++',
  '+-0+'],
 'pbl_12': ['---+',
  '---0',
  '----',
  '---0',
  '----',
  '+---',
  '++--',
  '++0-',
  '+++-'],
 'pbl_2': ['----', '0---', '+---', '++--', '++0-'],
 'pbl_3': ['-0-+',
  '---+',
  '-+--',
  '0--+',
  '00-0',
  '00-+',
  '0--+',
  '0+--',
  '0--+',
  '00-+',
  '0--+',
  '00-+',
  '0+-0',
  '+--+',
  '++-+',
  '+--+'],
 'pbl_4': ['---0',
  '----',
  '0---',
  '+---',
  '+0--',
  '++--',
  '++0-',
  '+++-',
  '+-+-'],
 'pbl_5': ['----', '0---', '+---', '+0--', '++--', '++0-'],
 'pbl_6': ['---0', '----', '0---', '+---', '+0--', '++--', '++0-', '+++-'],
 'pbl_7': ['----', '0---', '+0--', '++--', '++0-', '+-+-'],
 'pbl_8': ['---+',
  '+-+-',
  '+-+0',
  '+-++',
  '+++-',
  '+

In [14]:
with open("study_qtc_seqs.pickle", "w") as f:
    pickle.dump(qtc_seqs, f)

In [15]:
with open("study_qtc_seqs.pickle", "r") as f:
    print(pickle.load(f))

{'rotl_10': ['-+-+', '-0-+', '-+-+', '-0-+', '-+-+', '-0-+', '-+-+', '-0-+', '00-+', '0--+'], 'rotl_11': ['-+-+', '-0-+', '-+-+', '-0-+', '---+', '---0', '-0-0', '-0-+', '---+', '-0-+', '-+-+', '-0-+', '---+', '---0', '-0-0', '-0--', '00--', '00-0', '00-+', '0--+', '0--0', '00-0', '0+--', '+0-0', '+0--', '+0-0', '+--0', '+--+', '+--0', '+--+', '+0-+', '+--+', '+0-+', '+--+'], 'rotl_12': ['-0-+', '-+-+', '-0-+', '-+-+', '-0-+', '-+-+', '-0-+', '-+-+', '-0-+', '00-+', '0--+', '00-+'], 'pbl_10': ['----', '0---', '+---', '+0--', '++--', '++0-', '+++-', '+-+-'], 'pbl_11': ['----', '0---', '+0--', '++--', '++0-', '+-0-', '+-+-', '+-++', '+-+0', '+-++', '+-0+'], 'pbl_12': ['---+', '---0', '----', '---0', '----', '+---', '++--', '++0-', '+++-'], 'pbl_8': ['---+', '+-+-', '+-+0', '+-++', '+++-', '+-++', '+-+-', '+++-', '+-++', '+---', '+0--', '++--', '++0-', '+++-'], 'rotr_5': ['-+00', '-++0', '-++-', '-+0-', '-+00', '-++0', '-++-', '-++0', '-+++', '-++0', '-++-', '-++0', '-++-', '-++0', '-++-'